In [1]:
import tensorflow as tf
import tensorflow_hub as hub

import tensorflow_datasets as tfds

import time

from PIL import Image
import requests
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

import os
import pathlib

In [15]:
# パラメータ一覧
DATA_DIR = "C:\\tmp\\bit_image"

MODEL_DIR = '/tmp/my_saved_bit_model/'

DATA_SET_NAME = 'tf_flowers'

TRAIN_SPLIT = 0.9

NUM_CLASSES = 5

# 画像の解像度とデータセットの量でパラメータの指針がある。いったん軽いほうにしてるけど
# RESIZE_TOとCROP_TOはコメントアウトの方に変えてもいいかも
RESIZE_TO = 160 # 512
CROP_TO = 128 # 480

SCHEDULE_LENGTH = 500# 10000, 20000
SCHEDULE_BOUNDARIES = [200, 300, 400] #[3000, 6000, 9000] [6000, 12000, 18000]

# バッチサイズ512だとOOM発生するかも
# その場合2^nの形で修正して
BATCH_SIZE = 512
STEPS_PER_EPOCH = 10

# 今コメントアウトしてるけどこっちで学習させても十分かも
EPOCHS_NUM = 10

In [3]:
# ラベル一覧
tf_flowers_labels = ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [4]:
# モジュールのロード
model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
module = hub.KerasLayer(model_url)

In [5]:
# ローディングヘルパー関数

def preprocess_image(image):
  image = np.array(image)
  # reshape into shape [batch_size, height, width, num_channels]
  img_reshaped = tf.reshape(image, [1, image.shape[0], image.shape[1], image.shape[2]])
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  image = tf.image.convert_image_dtype(img_reshaped, tf.float32)  
  return image

def load_image_from_url(url):
  """Returns an image with shape [1, height, width, num_channels]."""
  response = requests.get(url)
  image = Image.open(BytesIO(response.content))
  image = preprocess_image(image)
  return image

In [6]:
# データセット取得

dataset_name = DATA_SET_NAME
ds, info = tfds.load(name=dataset_name, split=['train'],  with_info=True, data_dir=DATA_DIR)
ds = ds[0]
num_examples = info.splits['train'].num_examples


In [7]:
# トレーニング、テストデータに分解
num_train = int(TRAIN_SPLIT * num_examples)
ds_train = ds.take(num_train)
ds_test = ds.skip(num_train)

DATASET_NUM_TRAIN_EXAMPLES = num_examples

In [8]:
# BiTモデルクラス
class MyBiTModel(tf.keras.Model):
  
    def __init__(self, num_classes, module):
        super().__init__()

        self.num_classes = num_classes
        self.head = tf.keras.layers.Dense(num_classes, kernel_initializer='zeros')
        self.bit_model = module

    # predict関数を最大の選択肢だのindex出力するようにしてる。後でテストするときに視覚的にわかりやすいためだけどこの辺は仕様次第かな
    def predict(self, image):
        return tf.argmax(self(image)[0]).numpy()
    
    # tf,functionつけてみたけどあんま早くないかも。Bitモデル自体がtensorflowオブジェクトの計算に最適化されてなさそう。
    @tf.function
    def call(self, images):
        bit_embedding = self.bit_model(images)
        # 論文のコードだとsoftmaxかけられてなかったけどさすがにちがくね？ってことでsofmaxかけてる
        # あんま精度でなかったら外してもいいかも
        return tf.keras.activations.softmax(self.head(bit_embedding))

model = MyBiTModel(num_classes=NUM_CLASSES, module=module)

In [9]:

# データの前処理パイプライン
# 論文にこれやってって書いてあったので従っておいたほうがよさそう

SCHEDULE_LENGTH = SCHEDULE_LENGTH * 512 / BATCH_SIZE

def cast_to_tuple(features):
  return (features['image'], features['label'])
  
def preprocess_train(features):
  features['image'] = tf.image.random_flip_left_right(features['image'])
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.image.random_crop(features['image'], [CROP_TO, CROP_TO, 3])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

def preprocess_test(features):
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

pipeline_train = (ds_train
                  .shuffle(10000)
                  .repeat(int(SCHEDULE_LENGTH * BATCH_SIZE / DATASET_NUM_TRAIN_EXAMPLES * STEPS_PER_EPOCH) + 1 + 50)
                  .map(preprocess_train, num_parallel_calls=8)
                  .batch(BATCH_SIZE)
                  .map(cast_to_tuple) 
                  .prefetch(2))

pipeline_test = (ds_test.map(preprocess_test, num_parallel_calls=1)
                  .map(cast_to_tuple) 
                  .batch(BATCH_SIZE)
                  .prefetch(2))

In [10]:
# optimizerと損失関数の定義
# 損失関数がクロスエントロピーなのにもともと負の値が許されてたの意味が分からない

lr = 0.003 * BATCH_SIZE / 512 

lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=SCHEDULE_BOUNDARIES, 
                                                                   values=[lr, lr*0.1, lr*0.001, lr*0.0001])
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [11]:
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

# これだと50エポックやると思うから少なくしてもいいと思う
history = model.fit(
    pipeline_train,
    batch_size=BATCH_SIZE,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs= int(SCHEDULE_LENGTH / STEPS_PER_EPOCH),  # EPOCHS_NUM
    validation_data=pipeline_test 
)

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 1.1966 - accuracy: 0.7986 

KeyboardInterrupt: 

In [ ]:
# テストデータ１００個取り出して予測と正解比較してみる
for features in list(ds_test.take(100).as_numpy_iterator()):
    image = features['image']
    image = preprocess_image(image)
    image = tf.image.resize(image, [CROP_TO, CROP_TO])
    print(f"予測値：{model.predict(image)}, 正解：{features['label']}")

In [12]:
# モデルの保存①モデル丸ごと保存
export_module_dir = '/tmp/my_saved_bit_model/'
tf.saved_model.save(model, export_module_dir)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /tmp/my_saved_bit_model/assets


INFO:tensorflow:Assets written to: /tmp/my_saved_bit_model/assets


In [35]:
# モデルのロード①
# load_model1 =  hub.KerasLayer(export_module_dir, trainable=True)
load_model1 =  tf.keras.models.load_model(export_module_dir)

In [16]:
# モデルの保存②重み行列だけ保存（こっちが普通,早い）
model.save_weights(MODEL_DIR)

In [36]:
# モデルのロード②

# こっちの場合モデルを作ってあげないといけないのでロードに時間かかる。今回は省略
# model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
# module = hub.KerasLayer(model_url)

load_model2 = MyBiTModel(num_classes=NUM_CLASSES, module=module)
load_model2.load_weights(MODEL_DIR)

In [37]:
load_model2

In [42]:
# それぞれで5個予測させてみる
for features in list(ds_test.take(5).as_numpy_iterator()):
    image = features['image']
    image = preprocess_image(image)
    image = tf.image.resize(image, [CROP_TO, CROP_TO])
    # オーバーライドした関数はセーブしてないっぽいからせっかく作ったpredict関数が動かない。悲しい
    print(f"予測値：{load_model1.predict(image)}, 正解：{features['label']}")

予測値：[[2.4309322e-06 9.9999130e-01 3.0795096e-07 2.6889343e-06 3.2237870e-06]], 正解：1
予測値：[[4.8564107e-06 9.0731301e-05 9.9948090e-01 6.9052789e-05 3.5444336e-04]], 正解：2
予測値：[[7.0161917e-08 1.9877762e-05 7.2032555e-05 1.5147390e-05 9.9989283e-01]], 正解：4
予測値：[[1.3449710e-07 7.8095545e-06 9.9993968e-01 1.4462028e-05 3.7869184e-05]], 正解：2
予測値：[[2.7567528e-05 2.6586337e-04 9.6471548e-02 3.1265701e-04 9.0292233e-01]], 正解：4


In [38]:
for features in list(ds_test.take(5).as_numpy_iterator()):
    image = features['image']
    image = preprocess_image(image)
    image = tf.image.resize(image, [CROP_TO, CROP_TO])
    load_model2(image)
    print(f"予測値：{load_model2.predict(image)}, 正解：{features['label']}")

予測値：1, 正解：1
予測値：2, 正解：2
予測値：4, 正解：4
予測値：2, 正解：2
予測値：4, 正解：4
